In [1]:
import glob
import os, re
import tensorflow as tf
from sklearn.model_selection import train_test_split
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])                                   ", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

#     if len(sentence) > 17 : continue    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played, and it pleased the lord <end>',
 '<start> but you don t really care for music, do you? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth, the fifth <end>',
 '<start> the minor fall, the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor2 = tokenizer.texts_to_sequences(corpus)
    
    tensor=[[]]
    for i in tensor2:
        if len(i) < 16:
            tensor.append(i)

            
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    #print(tensor,tokenizer)

    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


src_input = tensor[:,:-1]
trg_input = tensor[:,1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         trg_input,
                                                         test_size=0.2,
                                                         random_state=1)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, trg_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

Source Train: (128244, 14)
Target Train: (128244, 14)


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [5]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)



optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)



Epoch 1/30
626/626 [==============================] - 105s 163ms/step - loss: 3.3692
Epoch 2/30
626/626 [==============================] - 106s 169ms/step - loss: 2.9350
Epoch 3/30
626/626 [==============================] - 107s 170ms/step - loss: 2.7522
Epoch 4/30
626/626 [==============================] - 107s 170ms/step - loss: 2.5033
Epoch 6/30
626/626 [==============================] - 107s 170ms/step - loss: 2.4035
Epoch 7/30
626/626 [==============================] - 107s 170ms/step - loss: 2.3115
Epoch 8/30
626/626 [==============================] - 107s 170ms/step - loss: 2.2250
Epoch 9/30
626/626 [==============================] - 107s 171ms/step - loss: 2.1442
Epoch 10/30
626/626 [==============================] - 107s 170ms/step - loss: 2.0674
Epoch 11/30
626/626 [==============================] - 107s 171ms/step - loss: 1.9945
Epoch 12/30
626/626 [==============================] - 107s 171ms/step - loss: 1.9253
Epoch 13/30
626/626 [==============================] - 107s 17

In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

generate_text(model, tokenizer, init_sentence="<start> boy", max_len=20)

'<start> boy don t try to front, uh, i <end> '